In [0]:
%sql
GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION bronze TO `antonyjerin.n@fourthsignal.com`;
GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION silver TO `antonyjerin.n@fourthsignal.com`;
GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION gold TO `antonyjerin.n@fourthsignal.com`;


In [0]:
# Required each time the cluster is restarted which should be only on the first noteebok as they run in order

tiers = ["bronze", "silver", "gold"]

adls_paths = {tier : f"abfss://{tier}@jerinsa.dfs.core.windows.net/" for tier in tiers}

bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]
 
dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)





In [0]:
import requests
import json
from datetime import  date,timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted or geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"


try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was succsessful
    response.raise_for_status()  # Raise HTTP Error for bad responses (4xxor 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data found for the specified date range.")
    else:
       # Specify the ALDS path
       file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

       # Save the JSON data
       json_data = json.dumps(data, indent=4)
       dbutils.fs.put(file_path, json_data, overwrite=True)
       print(f"Data saved to {file_path}")
except Exception as e:
    print(f"Error feching data from API : {e}")






In [0]:
data[0]


In [0]:
# Define your variables

output_data = {
    "start_date" : start_date.isoformat(),
    "end_date"   : end_date.isoformat(),
    "bronze_adls" : bronze_adls,
    "silver_adls" : silver_adls,
    "gold_adls" : gold_adls
    
}

dbutils.jobs.taskValues.set("bronze_output", output_data)
